In [14]:
import pandas as pd
import numpy as np
import re

In [15]:
PATH = '/content/drive/MyDrive/gh/dacon/techclf/DATA'

In [16]:
train_df = pd.read_csv(PATH + '/train.csv')
test_df = pd.read_csv(PATH + '/test.csv')

In [17]:
train_df.head()

,index,제출년도,사업명,사업_부처명,계속과제여부,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,label
0,0,2016,농업기초기반연구,농촌진흥청,신규,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,○ 새로운 해충분류군의 동정기술 개발 및 유입확산 추적,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,"○ 새로운 돌발 및 외래해충의 신속, 정확한 동정법 향상\n\n\n○ 돌발 및 외래...","뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",24
1,1,2019,이공학학술연구기반구축(R&D),교육부,신규,지역대학우수과학자지원사업(1년~5년),대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,최종목표: TRAIL 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구. 발굴된...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,"1) TRAIL 내성 특이적 표적분자를 발굴하고, 이를 이용한 TRAIL 효과 증진...","대장암,항암제 내성,세포사멸,유전자발굴","TRAIL,Colorectal cancer,TRAIL resistance,Apopt...",0
2,2,2016,중소기업기술혁신개발,중소기업청,신규,혁신기업기술개발,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n* 소비자 및 바...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,* 국내 독자적인 비목질계 셀룰로오스 자원의 파이버 및 부직포 제조 등의 기술 확보...,"기능성 샐룰로오스 파이버,천연섬유,기능성 부직포,뷰티&amp;케어 제품,미용 솜","functional cellulose fiber,natural fiber,funct...",0
3,3,2018,창업성장기술개발(R&D),중소벤처기업부,신규,창업사업화연계과제,소화기 암 진단용 분자영상 형광프로브 개발,# 암특이적 바이오마커 발굴 및 바이오마커에 대한 프로브 개발\n\n\n1) 소화...,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,# 암 진단기술의 차별성: 소화기 암 특이 프로브 개발\n\n\n- 최근 체외진단시...,"분자 진단,형광 조영제,프로브,항체,대장암","Molecular diagnosis,Fluorescence,probe,antibod...",0
4,4,2016,이공학개인기초연구지원,교육부,계속,기본연구지원,위암환자의 항암제반응예측을 위한 BRCA 발현검사,"수술이 불가능한 위암환자는 생존기간은 10개월 안팎에 지나지 않고, 항암화학요법에 ...",-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,-본 연구는 파라핀보관조직에서 in situ hybridization로 mRNA 및...,"BRCA,제자리부합법,조직미세배열,마이크로RNA,위암,항암제반응,젊은 연령/가족성 위암","BRCA,Insituhybridization,tissuemicroarray,micr...",0


In [18]:
test_df.head()

,index,제출년도,사업명,사업_부처명,계속과제여부,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드
0,174304,2016,경제협력권산업육성,산업통상자원부,신규,자동차융합부품,R-FSSW 기술 적용 경량 차체 부품 개발 및 품질 평가를 위한 64채널 C-SC...,○ 차체 점용접부의 품질 검사를 위한 64채널 무선 기반 C-Scan 탐촉자 개발\...,○ 1차년도\n\n . 개발 탐촉 시스템의 성능 평가 위한 표준 시편 제작 시...,○ 기술적 파급효과\n\n - 본 연구에서 개발된 R-FSSW 접합 기술은 기존 ...,"마찰교반점용접, 비파괴 검사, 초음파 탐상, 씨 스캔, 용접 품질 평가","Friction Stir Spot Welding, Non-destructive ev..."
1,174305,2018,개인기초연구(과기정통부)(R&D),과학기술정보통신부,계속,신진연구(총연구비5천이상~1.5억이하),다입자계를 묘사하는 편미분방정식에 대한 연구,자연계에는 입자의 개수가 아주 큰 다양한 다입자계가 존재한다. 이런 다입자계의 효...,연구과제1. 무한입자계의 동역학 / 작용소(operator) 방정식에 대한 연구\n...,본 연구는 물리학에서 중요한 대상인 다입자계를 묘사하는 모델방정식의 정당성을 보장하...,"다체계 방정식,동역학의 안정성,양자역학,고전역학,평균장 극한,고전극한,비상대론적 극한","many particle system,stability of dynamics,qua..."
2,174306,2017,개인기초연구(미래부),과학기술정보통신부,계속,자유공모,저출생체중아 가족을 위한 지역사회중심의 퇴원후 추후관리프로그램 개발,"저출생체중아는 정상 체중 출생아에 비하여 운동, 정신, 행동 및 정서발달장애가 빈번...",본 연구는 퇴원 후 저출생체중아의 퇴원 후 추후관리 프로그램 중 가정방문 모델과 가...,본 연구는 저출생체중아 가족 대상의 지역사회기반의 추후관리 사업에 대한 구체적인 모...,"저출생체중아,추후관리,조기개입,자조모임,가정방문","lowbirthweightinfant,followup,earlyinterventio..."
3,174307,2018,산학연협력기술개발(R&D),중소벤처기업부,신규,전략 협력(지역유망),폴리싱용 5축 CNC 정밀 마이크로 시스템 개발,최근 초정밀 공작기계 시장의 트렌드를 살펴보면 다음과 같다.\n\n\n(1) 자동차...,(1) 0.5~1㎛의 가공정밀도(면조도) 구현\n\n\n - 국내에서는 연삭기를 제...,(1) 가공 공정 및 소요시간 단축\n\n\n 초정밀금형 가공시 정삭 후 추가적...,"초정밀가공,폴리싱 가공,마이크로,경면가공,난삭재가공","superprecision,hiprecision,micro,mirror like f..."
4,174308,2019,이공학학술연구기반구축(R&D),교육부,계속,지역대학우수과학자지원사업(1년~5년),다결정재료의 미세조직구조를 고려한 확률론적 응력부식균열 생성예측모델 개발,본 연구의 최종목표는 다결정 금속재료의 미세조직구조를 고려한 확률론적 응력부식균열(...,"총 연구기간은 3년으로, 연차별 연구내용 및 범위는 다음과 같다.\n\n ∙ 1차...",본 연구의 중요성 및 활용가능성은 다음과 같다.\n\n ∙ 기존의 재료전체 ave...,"결정구조,응력부식균열,확률론적모델","Crystalline structure,Stress corrosion crackin..."


In [19]:
'사업_부처명, 과제명'
'사업_부처명, 과제명, 한글키워드'

train_new_df=train_df[['사업_부처명','과제명','label']]
test_new_df=test_df[['사업_부처명','과제명']]

In [20]:
train_new_df['text'] = train_new_df['사업_부처명']+' '+train_new_df['과제명']
test_new_df['text'] = test_new_df['사업_부처명']+' '+test_new_df['과제명']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
train_drop_df=train_new_df.drop(columns=['과제명','사업_부처명'])
test_drop_df=test_new_df.drop(columns=['과제명','사업_부처명'])

In [22]:
train_drop_df.head()

,label,text
0,24,농촌진흥청 유전정보를 활용한 새로운 해충 분류군 동정기술 개발
1,0,교육부 대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지...
2,0,중소기업청 비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케...
3,0,중소벤처기업부 소화기 암 진단용 분자영상 형광프로브 개발
4,0,교육부 위암환자의 항암제반응예측을 위한 BRCA 발현검사


In [23]:
test_drop_df.head()

,text
0,산업통상자원부 R-FSSW 기술 적용 경량 차체 부품 개발 및 품질 평가를 위한 6...
1,과학기술정보통신부 다입자계를 묘사하는 편미분방정식에 대한 연구
2,과학기술정보통신부 저출생체중아 가족을 위한 지역사회중심의 퇴원후 추후관리프로그램 개발
3,중소벤처기업부 폴리싱용 5축 CNC 정밀 마이크로 시스템 개발
4,교육부 다결정재료의 미세조직구조를 고려한 확률론적 응력부식균열 생성예측모델 개발


In [24]:
punct = "/-'?!,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ','\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text.strip()

cleaned_train_corpus = []
cleaned_test_corpus = []

for sent in train_drop_df['text']:
    cleaned_train_corpus.append(clean_punc(sent, punct, punct_mapping))
    
for sent in test_drop_df['text']:
    cleaned_test_corpus.append(clean_punc(sent, punct, punct_mapping))

In [25]:
def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        texts[i] = texts[i].replace('미래창조과학부',' ')
        texts[i] = texts[i].replace('교육부',' ')
        texts[i] = texts[i].replace('과학기술정보통신부',' ')
        texts[i] = texts[i].replace('중소벤처기업부',' ')
        texts[i] = texts[i].replace('산업통상자원부',' ')
        texts[i] = texts[i].replace('개발',' ')


        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        re_pattern = r'\d+'
        review = re.sub(re_pattern, ' number ', review)
        review = review.lower() #lower case
        review = re.sub(r'[^가-힣a-zA-Z]', ' ', review)
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        review = re.sub("[一-龥]",'', review)
        corpus.append(review)
    return corpus

In [26]:
train_drop_df['c_text'] = clean_text(cleaned_train_corpus)
test_drop_df['c_text'] = clean_text(cleaned_test_corpus)

In [27]:
train_drop_df=train_drop_df.drop(columns='text')
test_drop_df=test_drop_df.drop(columns='text')

In [28]:
train_drop_df.to_csv(PATH + '/drop_train.csv')
test_drop_df.to_csv(PATH + '/drop_test.csv')

In [29]:
train_drop_df

,label,c_text
0,24,농촌진흥청 유전정보를 활용한 새로운 해충 분류군 동정기술
1,0,대장암의 trail 내성 표적 인자 발굴 및 trail 반응 예측 유전자 지도 구축...
2,0,중소기업청 비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케...
3,0,소화기 암 진단용 분자영상 형광프로브
4,0,위암환자의 항암제반응예측을 위한 brca 발현검사
...,...,...
174299,0,혈관내막 증식을 치료하기 위한 표적분자 발굴
174300,0,삼킴곤란 개선을 위한 물성조절 식품시스템에서의 유동층과립 복합소재의 유변물성 변화 ...
174301,0,식물 유전체 정보 고도화 및 핵심 유전자 군 진화연구
174302,23,농촌진흥청 콩 유전자원 내건성 내습성 대량평가 체계구축 및 자원선발


In [30]:
test_drop_df

,c_text
0,r fssw 기술 적용 경량 차체 부품 및 품질 평가를 위한 number 채널 c ...
1,다입자계를 묘사하는 편미분방정식에 대한 연구
2,저출생체중아 가족을 위한 지역사회중심의 퇴원후 추후관리프로그램
3,폴리싱용 number 축 cnc 정밀 마이크로 시스템
4,다결정재료의 미세조직구조를 고려한 확률론적 응력부식균열 생성예측모델
...,...
43571,농림축산식품부 포장용 인디케이팅 잉크 제조기술 및 인쇄 공정 기술
43572,루시아 신속 정확 저렴한 현장진단용 혁신적 혈액진단
43573,iter 삼중수소 저장 공급 du용기 및 시험
43574,무결정립계 금속 초박막의 제조 및 이를 활용한 기초 연구


In [1]:
from collections import Counter
import pandas as pd

## local
PATH = '..'
## colab
PATH = '/content/drive/MyDrive/gh/dacon/techclf/DATA'

train = pd.read_csv(PATH + '/drop_train.csv', error_bad_lines=False)

df = pd.DataFrame()
#df_c = pd.DataFrame()

for i in range(0,46):
    train_topics = train['label'] == i
    train_topic = train[train_topics]['c_text']
    #train_topic_c = train[train_topics]['c_title']

    train_topic_list = []
    #train_c_topic_list = []

    for sentense in train_topic:
        for word in sentense.split():
            train_topic_list.append(word)
    cnt = Counter(train_topic_list)
    cnt = cnt.most_common()
    cnt = pd.DataFrame(cnt)
    df = pd.concat([df,cnt],axis=1)

    # for sentense in train_topic_c:
    #     for word in sentense.split():
    #         train_c_topic_list.append(word)
    # cnt_c = Counter(train_c_topic_list)
    # cnt_c = cnt_c.most_common()
    # cnt_c = pd.DataFrame(cnt_c)
    # df_c = pd.concat([df_c,cnt_c],axis=1)

df.columns = ['0','count','1','count','2','count','3','count','4','count','5','count','6','count','7','count','8','count','9','count','10','count','11','count','12','count','13','count','14','count','15','count','16','count','17','count','18','count','19','count','20','count','21','count','22','count','23','count','24','count','25','count','26','count','27','count','28','count','29','count','30','count','31','count','32','count','33','count','34','count','35','count','36','count','37','count','38','count','39','count','40','count','41','count','42','count','43','count','44','count','45','count']
#df_c.columns = ['IT과학_0','count','경제_1','count','사회_2','count','생활문화_3','count','세계_4','count','스포츠_5','count','정치_6','count']




In [3]:
df.to_csv(PATH + '/count/count.csv', index=False)
#df_c.to_csv(PATH + '/Preprocessing/count_c.csv', index=False)

In [4]:
df = pd.read_csv(PATH + '/count/count.csv', error_bad_lines=False)
#df_c = pd.read_csv(PATH + '/Preprocessing/count_c.csv', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head(50)

,0,count,1,count.1,2,count.2,3,count.3,4,count.4,5,count.5,6,count.6,7,count.7,8,count.8,9,count.9,10,count.10,11,count.11,12,count.12,13,count.13,14,count.14,15,count.15,16,count.16,17,count.17,18,count.18,19,count.19,...,26,count.26,27,count.27,28,count.28,29,count.29,30,count.30,31,count.31,32,count.32,33,count.33,34,count.34,35,count.35,36,count.36,37,count.37,38,count.38,39,count.39,40,count.40,41,count.41,42,count.42,43,count.43,44,count.44,45,count.45
0,개발,59971,개발,785.0,과학기술정보통신부,172.0,개발,94.0,산업통상자원부,32.0,개발,994.0,개발,53.0,개발,60.0,개발,225.0,개발,57.0,개발,367.0,개발,113.0,개발,166.0,개발,376.0,개발,1057.0,개발,89.0,개발,288.0,개발,93.0,개발,1030.0,개발,2945.0,...,농촌진흥청,192.0,개발,314.0,개발,278.0,개발,668.0,개발,138.0,및,361.0,기상청,144.0,해양수산부,256.0,해양수산부,346.0,해양수산부,161.0,보건복지부,471.0,및,129.0,산림청,236.0,개발,129.0,산림청,222.0,개발,46.0,개발,28.0,에너지,192.0,교육부,43.0,개발,547.0
1,및,49514,과학기술정보통신부,626.0,및,136.0,산업통상자원부,93.0,number,32.0,및,575.0,산업통상자원부,37.0,및,40.0,산업통상자원부,183.0,해양수산부,45.0,및,255.0,및,96.0,과학기술정보통신부,121.0,과학기술정보통신부,200.0,number,566.0,산업통상자원부,84.0,산업통상자원부,168.0,산업통상자원부,74.0,및,570.0,중소벤처기업부,2776.0,...,및,174.0,및,278.0,및,162.0,및,410.0,및,87.0,연구,277.0,개발,121.0,및,252.0,및,202.0,및,143.0,및,471.0,식품의약품안전처,128.0,및,143.0,및,104.0,및,218.0,산업통상자원부,29.0,저전력,24.0,개발,161.0,개발,40.0,및,468.0
2,과학기술정보통신부,36825,및,427.0,개발,101.0,number,61.0,개발,29.0,과학기술정보통신부,544.0,태양열,34.0,위한,31.0,및,138.0,number,33.0,과학기술정보통신부,192.0,number,57.0,number,82.0,및,183.0,및,557.0,number,64.0,및,165.0,및,66.0,number,511.0,및,1433.0,...,개발,116.0,교육부,153.0,교육부,106.0,위한,262.0,국토교통부,52.0,개발,251.0,및,99.0,연구,199.0,개발,185.0,개발,127.0,개발,447.0,연구,120.0,개발,112.0,산림청,95.0,연구,143.0,위한,28.0,및,19.0,과학기술정보통신부,140.0,과학기술정보통신부,31.0,number,257.0
3,교육부,30771,산업통상자원부,263.0,연구,90.0,및,55.0,및,26.0,number,505.0,number,22.0,number,26.0,number,97.0,및,28.0,위한,140.0,산업통상자원부,54.0,및,74.0,연료전지,175.0,과학기술정보통신부,535.0,수소,61.0,위한,139.0,에너지,51.0,산업통상자원부,499.0,number,1403.0,...,연구,94.0,환경부,133.0,위한,94.0,환경부,253.0,기술,49.0,기상청,238.0,연구,60.0,교육부,126.0,연구,92.0,연구,60.0,연구,226.0,number,102.0,연구,75.0,기술,41.0,개발,140.0,및,24.0,과학기술정보통신부,17.0,하베스팅,112.0,및,20.0,과학기술정보통신부,250.0
4,number,29290,위한,220.0,미래창조과학부,70.0,위한,42.0,수차,14.0,태양전지,423.0,및,21.0,중소벤처기업부,23.0,위한,97.0,산업통상자원부,22.0,교육부,138.0,기술,48.0,교육부,72.0,산업통상자원부,164.0,산업통상자원부,387.0,및,56.0,과학기술정보통신부,128.0,위한,44.0,위한,441.0,위한,885.0,...,number,57.0,위한,125.0,과학기술정보통신부,92.0,중소벤처기업부,224.0,과학기술정보통신부,45.0,과학기술정보통신부,187.0,위한,51.0,과학기술정보통신부,113.0,위한,65.0,대응기술,40.0,위한,220.0,개발,82.0,위한,57.0,연구,39.0,환경부,81.0,시스템,17.0,위한,17.0,교육부,112.0,이용한,19.0,미세먼지,249.0
5,연구,26701,원자력안전위원회,195.0,플라즈마,57.0,기술,26.0,중소벤처기업부,13.0,산업통상자원부,383.0,위한,17.0,산업통상자원부,22.0,교육부,50.0,이용한,21.0,기술,121.0,생산,46.0,수소,72.0,number,138.0,교육부,327.0,과학기술정보통신부,44.0,number,109.0,기반,37.0,과학기술정보통신부,290.0,중소기업청,775.0,...,위한,56.0,과학기술정보통신부,117.0,기술,91.0,과학기술정보통신부,197.0,위한,41.0,교육부,158.0,기술,48.0,개발,105.0,교육부,53.0,위한,38.0,과학기술정보통신부,211.0,위한,49.0,기술,31.0,교육부,35.0,교육부,65.0,태양광,15.0,시스템,15.0,및,106.0,미래창조과학부,15.0,교육부,224.0
6,위한,24410,원전,190.0,위한,48.0,가스터빈,23.0,수력,12.0,위한,319.0,이용한,16.0,연구,19.0,풍력발전기,47.0,과학기술정보통신부,16.0,생산,114.0,위한,41.0,위한,59.0,위한,121.0,위한,281.0,연구,29.0,중소벤처기업부,90.0,시스템,31.0,기술,277.0,친환경,767.0,...,농림축산식품부,45.0,연구,108.0,환경부,84.0,이용한,186.0,교육부,38.0,환경부,153.0,과학기술정보통신부,33.0,해양,62.0,이용한,43.0,연안침식,34.0,바이러스,153.0,식중독,43.0,기능성,27.0,산사태,30.0,위한,56.0,number,10.0,number,13.0,연구,105.0,연구,15.0,위한,203.0
7,중소벤처기업부,18779,기술,176.0,iter,32.0,과학기술정보통신부,22.0,kw급,12.0,태양광,310.0,과학기술정보통신부,15.0,과학기술정보통신부,15.0,해상풍력,45.0,파력발전,15.0,number,103.0,과학기술정보통신부,39.0,촉매,53.0,교육부,114.0,중소벤처기업부,228.0,위한,28.0,교육부,81.0,과학기술정보통신부,29.0,시스템,214.0,산업통상자원부,543.0,...,식품의약품안전처,43.0,기술,69.0,이용한,58.0,교육부,164.0,환경부,34.0,위한,111.0,기반,32.0,이용한,37.0,과학기술정보통신부,40.0,실용화,32.0,교육부,135.0,식품,41.0,활용한,16.0,위한,29.0,구축,52.0,과학기술정보통신부,9.0,스마트,10.0,위한,87.0,위한,14.0,연구,186.0
8,이용한,14322,미래창조과학부,141.0,기술,32.0,기술개발,14.0,프란시스,11.0,교육부,308.0,시스템,13.0,지열,14.0,과학기술정보통신부,44.0,위한,14.0,산업통상자원부,102.0,환경부,39.0,고효율,46.0,기술,97.0,연구,212.0,시스템,24.0,기술,80.0,실증,23.0,중소벤처기업부,182.0,이용한,453.0,...,잔류농약,32.0,국토교통부,66.0,국토교통부,53.0,기술

In [ ]:
#미래창조과학부
#교육부
#과학기술정보통신부
#중소벤처기업부
#개발
#산업통상자원부